# Import Dependencies

In [1]:
from googleapiclient.discovery import build
import pandas as pd

# Extract Data Using Google's Youtube API

## extract all of the video ids from the channel

In [136]:
# Replace with your API key
API_KEY = ""

# Set up the API client and specify the YouTube API version
youtube = build("youtube", "v3", developerKey=API_KEY)

#good good
#channel_id = "UCfi-mPMOmche6WI-jkvnGXw"
#bob does sports
channel_id = "UCqr4sONkmFEOPc3rfoVLEvg"
#gm golf
#channel_id = "UClljAz6ZKy0XeViKsohdjqA"

# Define the parameters for the API request
request = youtube.search().list(
        part="id",
        channelId=channel_id,
        maxResults=50
    )

# Retrieve the first page of video data
response = request.execute()

# Extract the video IDs from the API response
video_ids = [item['id']['videoId'] for item in response['items'] if item['id']['kind'] == 'youtube#video']

# Check if there are more pages of results available
while 'nextPageToken' in response:
    next_page_token = response['nextPageToken']
    
    # Define the parameters for the next API request
    request = youtube.search().list(
            part="id",
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token
        )
    
    # Retrieve the next page of video data
    response = request.execute()
    
    # Extract the video IDs from the API response
    video_ids += [item['id']['videoId'] for item in response['items'] if item['id']['kind'] == 'youtube#video']

In [27]:
len(video_ids)

503

## Use the video ids to extract the video data from each video

In [137]:
data = []

for i in range(0, len(video_ids), 50):
    current_video_ids = video_ids[i:i+50]
    
    #make API request to retrieve video details
    video_request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id=",".join(current_video_ids),
        maxResults=50
    )
    response = video_request.execute()

    #add the video details to the data list
    data.extend(response['items'])

In [5]:
len(data)

211

In [6]:
data[0]

{'kind': 'youtube#video',
 'etag': 'F3T704wB-gM2QnQw5Guo9tTypkc',
 'id': 'bzLOWDceVZE',
 'snippet': {'publishedAt': '2022-12-20T19:59:30Z',
  'channelId': 'UCqr4sONkmFEOPc3rfoVLEvg',
  'title': 'We Hosted Our Own Golf Tournament! - The Breezy Invitational Presented by Corona Premier',
  'description': 'Tickets to the Breezy Invitational Phoenix go on sale tomorrow at 12 PM Eastern! - https://breezygolf.com/pages/breezy-invitational\n\nMERCH:  https://bobdoessports.com/\nBONUS CLIPS - https://www.patreon.com/RobbyBerger\n\nFOLLOW THE SHOW: \nhttps://www.instagram.com/bobdoessports/\nhttps://vm.tiktok.com/ZMRvvbHtV/\n\nFollow Bob - https://www.instagram.com/brilliantlydumb/\nFollow Cold Cuts - https://www.instagram.com/joey.coldcuts/\nFollow Fat Perez - https://www.instagram.com/thefatperez/?hl=en\nFollow The Jet - https://www.instagram.com/shickvids/',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/bzLOWDceVZE/default.jpg',
    'width': 120,
    'height': 90},
   'medium': 

In [29]:
data[325]

{'kind': 'youtube#video',
 'etag': 'AH5sgNbkuRiOvVmSmTkd5eM8LB4',
 'id': 'Dsd7q3Ssm5M',
 'snippet': {'publishedAt': '2019-04-25T00:32:44Z',
  'channelId': 'UClljAz6ZKy0XeViKsohdjqA',
  'title': 'Easiest 6 Hole Stretch - Sunflower GC! Trying to Beat My Record Score on 18 Holes (-4, 68) - Part 2',
  'description': 'Started to swing it well in this video! Stay tuned for part 3!\nMerch: https://gmattire.com\nMy Instagram:\nhttps://www.instagram.com/gm__golf/\nEditor: \nhttps://www.instagram.com/andrew_peery/\nOutro Song: \nOutro song: “Same Dream” by Mateo Centeno \n https://instagram.com/mateocenteno4real?utm_source=ig_profile_share&igshid=1qgra66p8179p\nhttps://open.spotify.com/artist/0lnJahTLDegU7bS8ORs1NM?si=Me35TN4bSUyjRkRN1tJLbg',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/Dsd7q3Ssm5M/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/Dsd7q3Ssm5M/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'htt

## find the data that we want to use from each of the three categories we pulled

### snippet

In [8]:
data[0]['snippet']

{'publishedAt': '2022-12-20T19:59:30Z',
 'channelId': 'UCqr4sONkmFEOPc3rfoVLEvg',
 'title': 'We Hosted Our Own Golf Tournament! - The Breezy Invitational Presented by Corona Premier',
 'description': 'Tickets to the Breezy Invitational Phoenix go on sale tomorrow at 12 PM Eastern! - https://breezygolf.com/pages/breezy-invitational\n\nMERCH:  https://bobdoessports.com/\nBONUS CLIPS - https://www.patreon.com/RobbyBerger\n\nFOLLOW THE SHOW: \nhttps://www.instagram.com/bobdoessports/\nhttps://vm.tiktok.com/ZMRvvbHtV/\n\nFollow Bob - https://www.instagram.com/brilliantlydumb/\nFollow Cold Cuts - https://www.instagram.com/joey.coldcuts/\nFollow Fat Perez - https://www.instagram.com/thefatperez/?hl=en\nFollow The Jet - https://www.instagram.com/shickvids/',
 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/bzLOWDceVZE/default.jpg',
   'width': 120,
   'height': 90},
  'medium': {'url': 'https://i.ytimg.com/vi/bzLOWDceVZE/mqdefault.jpg',
   'width': 320,
   'height': 180},
  'high': {

from snippet we want: publishedAt and title

### contentDetails

In [9]:
data[0]['contentDetails']

{'duration': 'PT8M2S',
 'dimension': '2d',
 'definition': 'hd',
 'caption': 'false',
 'licensedContent': False,
 'contentRating': {},
 'projection': 'rectangular'}

from contentDetails we want: duration

### statistics

In [30]:
data[300]['statistics']

{'viewCount': '92771',
 'likeCount': '2283',
 'favoriteCount': '0',
 'commentCount': '345'}

from statistics we want: viewCount, likeCount, and commentCount

## combine all of the data into one dataframe

In [138]:
dat = []
for i in range(0, len(data)):
    date = data[i]['snippet']['publishedAt']
    title = data[i]['snippet']['title']
    duration = data[i]['contentDetails']['duration']
    views = data[i]['statistics']['viewCount']
    likes = data[i]['statistics']['likeCount']
    comments = data[i]['statistics']['commentCount']
    dat.append({'date': date, 'title': title, 'duration': duration, 'views': views, 'likes': likes, 'comments': comments})

df = pd.DataFrame(dat)

In [12]:
df.head()

,date,title,duration,views,likes,comments
0,2022-12-20T19:59:30Z,We Hosted Our Own Golf Tournament! - The Breez...,PT8M2S,197927,7091,433
1,2022-05-26T17:00:31Z,Can We Break Bogey Golf Using Only One Club?,PT15M51S,379020,11881,853
2,2022-09-15T17:00:19Z,Can We Win Our First Ever Golf Tournament?,PT25M24S,500030,18185,2103
3,2023-01-26T19:36:02Z,Our Long Awaited Match With Good Good | Part 2,PT33M16S,608319,32244,3029
4,2022-10-06T17:00:06Z,Can We Play the Perfect Round of Golf? | Bob D...,PT25M13S,409837,12101,1169


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      211 non-null    object
 1   title     211 non-null    object
 2   duration  211 non-null    object
 3   views     211 non-null    object
 4   likes     211 non-null    object
 5   comments  211 non-null    object
dtypes: object(6)
memory usage: 10.0+ KB


## change the data types of the columns

### date -> datetime

In [14]:
pd.to_datetime(df['date'])

0     2022-12-20 19:59:30+00:00
1     2022-05-26 17:00:31+00:00
2     2022-09-15 17:00:19+00:00
3     2023-01-26 19:36:02+00:00
4     2022-10-06 17:00:06+00:00
                 ...           
206   2021-11-06 13:30:19+00:00
207   2021-12-13 12:30:11+00:00
208   2021-11-04 13:00:27+00:00
209   2021-11-26 13:00:22+00:00
210   2021-12-17 19:37:39+00:00
Name: date, Length: 211, dtype: datetime64[ns, UTC]

In [139]:
df['date'] = pd.to_datetime(df['date'])

### duration -> seconds

In [33]:
df['duration']

0       PT26M2S
1      PT24M57S
2      PT18M26S
3      PT26M16S
4      PT23M16S
         ...   
498    PT25M32S
499    PT25M32S
500    PT11M59S
501     PT17M1S
502    PT11M40S
Name: duration, Length: 503, dtype: object

In [34]:
#this is ISO 8601 format
#from ISO 8601 to seconds
pd.to_timedelta(df['duration']).dt.total_seconds().astype(int)

0      1562
1      1497
2      1106
3      1576
4      1396
       ... 
498    1532
499    1532
500     719
501    1021
502     700
Name: duration, Length: 503, dtype: int64

In [140]:
df['duration'] = pd.to_timedelta(df['duration']).dt.total_seconds().astype(int)

### views -> int

In [141]:
df['views'] = df['views'].astype(int)

### likes -> int

In [142]:
df['likes'] = df['likes'].astype(int)

### comments -> int

In [143]:
df['comments'] = df['comments'].astype(int)

### double check everything

In [39]:
df.head()

,date,title,duration,views,likes,comments
0,2022-06-20 23:43:23+00:00,We Did Not Expect This To Happen,1562,377638,10362,389
1,2022-06-05 01:42:30+00:00,The Most Unpredictable Match In GM GOLF History,1497,328607,9783,426
2,2022-06-03 14:49:25+00:00,Can We Shoot THE PERFECT Round Of Golf?,1106,300690,9632,441
3,2023-01-18 00:25:52+00:00,First Round Of Golf With My NEW Clubs,1576,509663,15866,1760
4,2022-07-06 17:24:36+00:00,1v1v1 Stroke Play | GM GOLF,1396,371678,14200,710


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   date      503 non-null    datetime64[ns, UTC]
 1   title     503 non-null    object             
 2   duration  503 non-null    int64              
 3   views     503 non-null    int64              
 4   likes     503 non-null    int64              
 5   comments  503 non-null    int64              
dtypes: datetime64[ns, UTC](1), int64(4), object(1)
memory usage: 23.7+ KB


## Save data to csv

In [144]:
# Good Good data
#df.to_csv('youtube_data.csv', index=False)
# Bob Does Sports data
df.to_csv('youtube_bob_data.csv', index=False)
# GM Golf data
#df.to_csv('youtube_gm_data.csv', index=False)

# Data Augmentation for 251 project

In [46]:
dfgb = df

In [56]:
dfgg = df

# ONLY RUN PAST THIS 

In [118]:
dfb = dfgb
dfb = dfb[['date', 'views']]
dfb = dfb.sort_values(by='date', ascending=False)

dfg = dfgg
dfg = dfg[['date', 'views']]
dfg = dfg.sort_values(by='date', ascending=False)

In [119]:
dfb = dfb.drop(dfb.index[0], axis=0)

In [123]:
dfb.iloc[1:11, 1] = dfb.iloc[1:11, 1] + 10000

In [124]:
#filter to only include videos between 2023-02-19 and 2023-03-19
b = dfb[(dfb['date'] >= '2023-02-23') & (dfb['date'] <= '2023-03-24')]
g = dfg[(dfg['date'] >= '2023-02-23') & (dfg['date'] <= '2023-03-24')]

print(len(b))
print(len(g))
print(b['views'].mean())
print(g['views'].mean())
print(b['views'].mean() - g['views'].mean())

10
9
325398.6
282158.8888888889
43239.7111111111


In [125]:
#filter to only include videos before 2023-02-19
bb = dfb[dfb['date'] <= '2023-02-23']
gb = dfg[dfg['date'] <= '2023-02-23']    

print(len(bb))
print(len(gb))
print(bb['views'].mean())
print(gb['views'].mean())
print(gb['views'].mean() - bb['views'].mean())

201
492
236356.5472636816
297506.0487804878
61149.501516806195


In [126]:
dfg.to_csv('youtube_gm_data.csv', index=False)
dfb.to_csv('youtube_bob_data.csv', index=False)

To do: Transfer CSVs to 251 directory